In [ ]:
# nuclio: start-code

In [12]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd

In [46]:
def handler(context):
    context.logger.info("Reading features from feature vector")
    stock_list = []
    service = fs.get_online_feature_service("stocks-vec")
    for key,value in context.sym_to_url.items():
        stock_list += service.get([{"symbol": key}],as_list = True)
    df = pd.DataFrame(stock_list,index = [item[0] for item in context.sym_to_url.items()])
    df.index.name = "symbol"
    df.columns = ["price","volume","last_updated","sentiment"]
    return df

In [18]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    mlrun.set_environment(project="stocks-avia")
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    setattr(context, 'news', "news")
    setattr(context, 'stocks', "stocks")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.price",
               "stocks.volume",
               "stocks.last_updated",
               "news.sentiment"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [15]:
# nuclio: end-code

In [59]:
# test local
init_context(context)
df = handler(context)

Python> 2021-03-21 06:24:07,970 [info] Initalizing context & constructing featrure vector
Python> 2021-03-21 06:24:08,025 [info] Reading features from feature vector


In [76]:
df

,price,volume,last_updated,sentiment
symbol,,,,
GOOGL,2026.959961,61164.0,2021-03-19 15:59:00-04:00,-1.333333
MSFT,229.910004,656799.0,2021-03-19 15:59:00-04:00,-1.666667
AMZN,3074.959961,113128.0,2021-03-19 15:59:00-04:00,-2.000000
AAPL,119.870003,2036817.0,2021-03-19 15:59:00-04:00,3.000000


In [77]:
stock_feature_set = fs.FeatureSet("stocks_fs", entities=[fs.Entity("symbol")])

In [78]:
stock_feature_set.add_aggregation("sentiments","sentiment",["min","max"],["1h"],"10m")
stock_feature_set.add_aggregation("prices","price",["min","max","sum"],["1h"],"10m")
stock_feature_set.add_aggregation("volumes","volume",["min","max"],["1h"],"10m")

In [79]:
fs.infer_metadata(
    stock_feature_set,
    df,
    entity_columns=["symbol"],
    options=fs.InferOptions.default(),
)

,sentiments_max_1h,sentiments_min_1h,prices_max_1h,prices_min_1h,prices_sum_1h,volumes_max_1h,volumes_min_1h,price,volume,last_updated,sentiment
symbol,,,,,,,,,,,
GOOGL,-1.333333,-1.333333,2026.959961,2026.959961,2026.959961,61164.0,61164.0,2026.959961,61164.0,2021-03-19 15:59:00-04:00,-1.333333
MSFT,-1.666667,-1.666667,229.910004,229.910004,229.910004,656799.0,656799.0,229.910004,656799.0,2021-03-19 15:59:00-04:00,-1.666667
AMZN,-2.000000,-2.000000,3074.959961,3074.959961,3074.959961,113128.0,113128.0,3074.959961,113128.0,2021-03-19 15:59:00-04:00,-2.000000
AAPL,3.000000,3.000000,119.870003,119.870003,119.870003,2036817.0,2036817.0,119.870003,2036817.0,2021-03-19 15:59:00-04:00,3.000000
